#### [`SageMathCell Version`](https://olgabelitskaya.github.io/sql_cookbook.html) & [`SageMathCell Test Example`](https://olgabelitskaya.github.io/sql_test.html)
## 📑 Creating SQL Databases

In [1]:
import zipfile,sqlite3,os; import pandas as pd
import numpy as np,sympy as sp,pylab as pl
import warnings; warnings.filterwarnings("ignore")
from IPython.core.display import display,HTML
from IPython.core.magic import register_line_magic
@register_line_magic
def get_query(q):
    sp.pprint(r'SQL Queries')
    tr=[]; cursor.execute(q)
    result=cursor.fetchall()
    for r in result: 
        tr+=[r]
    display(pd.DataFrame.from_records(tr)\
              .style.set_table_styles(style_dict))
def connect_to_db(dbf):
    sqlconn=None
    try:
        sqlconn=sqlite3.connect(dbf)
        return sqlconn
    except Error as err:
        print(err)
        if sqlconn is not None:
            sqlconn.close()
connection=connect_to_db('example.db')
if connection is not None:
    cursor=connection.cursor()
thp=[('font-size','15px'),('text-align','center'),
     ('font-weight','bold'),('padding','5px 5px'),
     ('color','white'),('background-color','slategray')]
tdp=[('font-size','14px'),('padding','5px 5px'),
     ('text-align','center'),('color','darkblue'),
     ('background-color','silver')]
style_dict=[dict(selector='th',props=thp),
            dict(selector='td',props=tdp)]
[os.listdir(),os.listdir('../input'),
 os.listdir('../input/data-science-for-good')]

[['example.db', '__notebook__.ipynb'],
 ['data-science-for-good', 'nba-all-star-game-20002016'],
 ['D5 SHSAT Registrations and Testers.csv', '2016 School Explorer.csv']]

## 📑 Create SQL Tables

In [2]:
df1=pd.DataFrame({'key':['A','B','C','D','F','F'],
                  'value':np.random.randn(6)})
df2=pd.DataFrame({'key':['B','D','D','E','F'],
                  'value':np.random.randn(5)})
for df in [df1,df2]:
    display(df.style.set_table_styles(style_dict))

,key,value
0,A,-0.152623
1,B,1.02335
2,C,1.23849
3,D,-0.458265
4,F,0.524034
5,F,0.581224


,key,value
0,B,-0.259452
1,D,0.0685188
2,D,-0.566746
3,E,-1.65508
4,F,-0.701908


In [3]:
# from pandas dataframes
df1.to_sql('df1',con=connection,
            index=False,if_exists='replace')
df2.to_sql('df2',con=connection,
            index=False,if_exists='replace')

In [4]:
# using the sqlite3 module
cursor.execute('''
CREATE TABLE IF NOT EXISTS projects (
id integer PRIMARY KEY,
name text NOT NULL,
begin_date text,
end_date text);
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS tasks (
id integer PRIMARY KEY,
name text NOT NULL,
priority integer,
status_id integer NOT NULL,
project_id integer NOT NULL,
begin_date text NOT NULL,
end_date text NOT NULL,
FOREIGN KEY (project_id) REFERENCES projects (id)
);
''')

In [5]:
def create_project(cursor,project):
    q='''INSERT INTO projects(name,begin_date,end_date)
         VALUES(?,?,?)'''
    cursor.execute(q,project)
    return cursor.lastrowid
def create_task(cursor,task):
    q='''INSERT INTO tasks(
         name,priority,status_id,
         project_id,begin_date,end_date
         ) VALUES(?,?,?,?,?,?)'''
    cursor.execute(q,task)
    return cursor.lastrowid
project1=('SQL CookBook with SageMathCell',
          '2020-02-04','2020-02-18')
project2=('SQL Tests',
          '2020-01-31','2020-02-14')
project_id1=create_project(cursor,project1)
project_id2=create_project(cursor,project2)
t=[('Page 1',int(1),int(1),
    project_id1,'2020-02-04','2020-02-12'),
   ('Page 2',int(1),int(1),
    project_id1,'2020-02-10','2020-02-18'),
   ('Test 1',int(1),int(1),
    project_id2,'2020-01-31','2020-02-01'),
   ('Test 2',int(1),int(1),
    project_id2,'2020-02-02','2020-02-03'),
   ('Test 3',int(1),int(1),
    project_id2,'2020-02-04','2020-02-05'),
   ('Test 4',int(1),int(1),
    project_id2,'2020-02-06','2020-02-07'),
   ('Test 5',int(1),int(1),
    project_id2,'2020-02-08','2020-02-09'),
   ('Test 6',int(1),int(1),
    project_id2,'2020-02-09','2020-02-10'),
   ('Test 7',int(1),int(1),
    project_id2,'2020-02-10','2020-02-11'),
   ('Dublicate',int(1),int(1),
    project_id2,'2020-02-11','2020-02-14')]
for task in t:        
    create_task(cursor,task)

In [6]:
# from pandas dataframes and csv files (external files)
url='https://raw.github.com/pydata/pandas/'+\
    'master/pandas/tests/data/tips.csv'
tips=pd.read_csv(url)
tips.to_sql('tips',con=connection,
            if_exists='replace')
tips.head(int(10)).style\
    .set_table_styles(style_dict)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.5,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4
8,15.04,1.96,Male,No,Sun,Dinner,2
9,14.78,3.23,Male,No,Sun,Dinner,2


In [7]:
# from pandas dataframes and csv files (internal files)
url2='../input/data-science-for-good/2016 School Explorer.csv'
se=pd.read_csv(url2)
percent_list = ['Percent ELL','Percent Asian','Percent Black',
                'Percent Hispanic','Percent Black / Hispanic',
                'Percent White','Student Attendance Rate',
                'Percent of Students Chronically Absent',
                'Rigorous Instruction %','Collaborative Teachers %',
                'Supportive Environment %',
                'Effective School Leadership %',
                'Strong Family-Community Ties %','Trust %']
geo_list=['Location Code','District']
target_list=['Average ELA Proficiency','Average Math Proficiency']
economic_list=['Economic Need Index','School Income Estimate']
rating_list=['Rigorous Instruction Rating','Collaborative Teachers Rating',
             'Supportive Environment Rating','Effective School Leadership Rating',
             'Strong Family-Community Ties Rating','Trust Rating',
             'Student Achievement Rating']
se=se[geo_list+percent_list+target_list+economic_list+rating_list].dropna()
se['School Income Estimate']=se['School Income Estimate'].astype('str') 
for s in [",","$"," "]:
    se['School Income Estimate']=se['School Income Estimate'].str.replace(s,"")
se['School Income Estimate']=se['School Income Estimate'].astype(float)
for el in percent_list:
    se[el]=se[el].astype('str')
    se[el]=se[el].str.replace("%","")
    se[el]=se[el].astype(float)
se.columns=se.columns.str.replace(' ','')
se.to_sql('schooledu',con=connection,
          if_exists='replace')
se.head(3).T.style.set_table_styles(style_dict)

,0,2,3
LocationCode,01M015,01M020,01M034
District,1,1,1
PercentELL,9,15,7
PercentAsian,5,35,5
PercentBlack,32,8,29
PercentHispanic,60,49,63
PercentBlack/Hispanic,92,57,92
PercentWhite,1,4,4
StudentAttendanceRate,94,94,92
PercentofStudentsChronicallyAbsent,18,20,28


In [8]:
# from pandas dataframes and csv files (internal files)
url3='../input/nba-all-star-game-20002016/NBA All Stars 2000-2016 - Sheet1.csv'
nba=pd.read_csv(url3).dropna()
nba.columns=nba.columns.str.replace(' ','')
nba.to_sql('nba',con=connection,
          if_exists='replace')
nba.head().T.style.set_table_styles(style_dict)

,0,1,2,3,4
Year,2016,2016,2016,2016,2016
Player,Stephen Curry,James Harden,Kevin Durant,Kawhi Leonard,Anthony Davis
Pos,G,SG,SF,F,PF
HT,6-3,6-5,6-9,6-7,6-11
WT,190,220,240,230,253
Team,Golden State Warriors,Houston Rockets,Golden State Warriors,San Antonio Spurs,New Orleans Pelicans
SelectionType,Western All-Star Fan Vote Selection,Western All-Star Fan Vote Selection,Western All-Star Fan Vote Selection,Western All-Star Fan Vote Selection,Western All-Star Fan Vote Selection
NBADraftStatus,2009 Rnd 1 Pick 7,2009 Rnd 1 Pick 3,2007 Rnd 1 Pick 2,2011 Rnd 1 Pick 15,2012 Rnd 1 Pick 1
Nationality,United States,United States,United States,United States,United States


## 📑 SQL Basic Queries

In [9]:
%get_query \
PRAGMA table_info("schooledu")

SQL Queries


,0,1,2,3,4,5
0,0,index,INTEGER,0,None,0
1,1,LocationCode,TEXT,0,None,0
2,2,District,INTEGER,0,None,0
3,3,PercentELL,REAL,0,None,0
4,4,PercentAsian,REAL,0,None,0
5,5,PercentBlack,REAL,0,None,0
6,6,PercentHispanic,REAL,0,None,0
7,7,PercentBlack/Hispanic,REAL,0,None,0
8,8,PercentWhite,REAL,0,None,0
9,9,StudentAttendanceRate,REAL,0,None,0


In [10]:
pq=pd.read_sql_query('''
SELECT *
FROM df1,df2;
''',con=connection)
pq.set_axis(['key1','value1','key2','value2'],
            axis=1,inplace=True)
pq.style.set_table_styles(style_dict)

,key1,value1,key2,value2
0,A,-0.152623,B,-0.259452
1,A,-0.152623,D,0.0685188
2,A,-0.152623,D,-0.566746
3,A,-0.152623,E,-1.65508
4,A,-0.152623,F,-0.701908
5,B,1.02335,B,-0.259452
6,B,1.02335,D,0.0685188
7,B,1.02335,D,-0.566746
8,B,1.02335,E,-1.65508
9,B,1.02335,F,-0.701908


In [11]:
pq=pd.read_sql_query('''
SELECT *
FROM df1,df2
WHERE df1.key=df2.key AND df1.value>0
ORDER BY key DESC;
''',con=connection)
pq.set_axis(['key1','value1','key2','value2'],
            axis=1,inplace=True)
pq.style.set_table_styles(style_dict)

,key1,value1,key2,value2
0,F,0.524034,F,-0.701908
1,F,0.581224,F,-0.701908
2,B,1.02335,B,-0.259452


In [12]:
%get_query \
SELECT * \
FROM projects,tasks; 

SQL Queries


,0,1,2,3,4,5,6,7,8,9,10
0,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,1,Page 1,1,1,1,2020-02-04,2020-02-12
1,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,2,Page 2,1,1,1,2020-02-10,2020-02-18
2,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,3,Test 1,1,1,2,2020-01-31,2020-02-01
3,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,4,Test 2,1,1,2,2020-02-02,2020-02-03
4,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,5,Test 3,1,1,2,2020-02-04,2020-02-05
5,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,6,Test 4,1,1,2,2020-02-06,2020-02-07
6,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,7,Test 5,1,1,2,2020-02-08,2020-02-09
7,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,8,Test 6,1,1,2,2020-02-09,2020-02-10
8,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,9,Test 7,1,1,2,2020-02-10,2020-02-11
9,1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,10,Dublicate,1,1,2,2020-02-11,2020-02-14


In [13]:
%get_query \
SELECT COUNT(LocationCode),TrustRating  \
FROM schooledu \
GROUP BY TrustRating;

SQL Queries


,0,1
0,124,Approaching Target
1,263,Exceeding Target
2,384,Meeting Target
3,24,Not Meeting Target


In [14]:
%get_query \
SELECT DISTINCT Team  \
FROM nba \
WHERE Year=2014;

SQL Queries


,0
0,Cleveland Cavaliers
1,Washington Wizards
2,Toronto Raptors
3,Chicago Bulls
4,New York Knicks
5,Atlanta Hawks
6,Miami Heat
7,Golden State Warriors
8,Los Angeles Lakers
9,New Orleans Pelicans


## 📑  SQL & Pandas Similarity

In [15]:
sp.pretty_print('Pandas Queries')
display(tips[['total_bill','tip','smoker','time']].head(int(7))\
.style.set_table_styles(style_dict))
%get_query \
SELECT total_bill,tip,smoker,time \
FROM tips \
LIMIT 7;

Pandas Queries


,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.5,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner
5,25.29,4.71,No,Dinner
6,8.77,2,No,Dinner


SQL Queries


,0,1,2,3
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.5,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner
5,25.29,4.71,No,Dinner
6,8.77,2,No,Dinner


In [16]:
sp.pretty_print('Pandas Queries')
display(tips[tips['time']=='Dinner'].head(int(3))\
.style.set_table_styles(style_dict))
%get_query \
SELECT * \
FROM tips \
WHERE time='Dinner' \
LIMIT 3;

Pandas Queries


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.5,Male,No,Sun,Dinner,3


SQL Queries


,0,1,2,3,4,5,6,7
0,0,16.99,1.01,Female,No,Sun,Dinner,2
1,1,10.34,1.66,Male,No,Sun,Dinner,3
2,2,21.01,3.5,Male,No,Sun,Dinner,3


In [17]:
sp.pretty_print('Pandas Queries')
display(tips.groupby(['day','sex'])\
.agg({'tip':np.mean,'day':np.size})\
.style.set_table_styles(style_dict))
%get_query \
SELECT day,sex,AVG(tip),COUNT(*) \
FROM tips \
GROUP BY day,sex;

Pandas Queries


SQL Queries


,0,1,2,3
0,Fri,Female,2.78111,9
1,Fri,Male,2.693,10
2,Sat,Female,2.80179,28
3,Sat,Male,3.0839,59
4,Sun,Female,3.36722,18
5,Sun,Male,3.22034,58
6,Thur,Female,2.57563,32
7,Thur,Male,2.98033,30


In [18]:
sp.pretty_print('Pandas Queries')
display(pd.merge(df1,df2,on='key')\
.style.set_table_styles(style_dict))
%get_query \
SELECT * \
FROM df1 \
INNER JOIN df2 \
ON df1.key=df2.key;

Pandas Queries


,key,value_x,value_y
0,B,1.02335,-0.259452
1,D,-0.458265,0.0685188
2,D,-0.458265,-0.566746
3,F,0.524034,-0.701908
4,F,0.581224,-0.701908


SQL Queries


,0,1,2,3
0,B,1.02335,B,-0.259452
1,D,-0.458265,D,-0.566746
2,D,-0.458265,D,0.0685188
3,F,0.524034,F,-0.701908
4,F,0.581224,F,-0.701908


In [19]:
sp.pretty_print('Pandas Queries')
display(pd.merge(df1,df2,on='key',how='left')\
.style.set_table_styles(style_dict))
%get_query \
SELECT * \
FROM df1 \
LEFT OUTER JOIN df2 \
ON df1.key=df2.key;

Pandas Queries


,key,value_x,value_y
0,A,-0.152623,nan
1,B,1.02335,-0.259452
2,C,1.23849,nan
3,D,-0.458265,0.0685188
4,D,-0.458265,-0.566746
5,F,0.524034,-0.701908
6,F,0.581224,-0.701908


SQL Queries


,0,1,2,3
0,A,-0.152623,None,nan
1,B,1.02335,B,-0.259452
2,C,1.23849,None,nan
3,D,-0.458265,D,-0.566746
4,D,-0.458265,D,0.0685188
5,F,0.524034,F,-0.701908
6,F,0.581224,F,-0.701908


In [20]:
sp.pretty_print('Pandas Queries')
display(pd.concat([df1,df2]).set_index(pd.Index(range(11)))\
.style.set_table_styles(style_dict))
%get_query \
SELECT key,value \
FROM df1 \
UNION ALL \
SELECT key,value \
FROM df2;

Pandas Queries


,key,value
0,A,-0.152623
1,B,1.02335
2,C,1.23849
3,D,-0.458265
4,F,0.524034
5,F,0.581224
6,B,-0.259452
7,D,0.0685188
8,D,-0.566746
9,E,-1.65508


SQL Queries


,0,1
0,A,-0.152623
1,B,1.02335
2,C,1.23849
3,D,-0.458265
4,F,0.524034
5,F,0.581224
6,B,-0.259452
7,D,0.0685188
8,D,-0.566746
9,E,-1.65508


## 📑 Close the Connection and Delete This Database

In [21]:
pd.read_sql_query('''
SELECT * from sqlite_master;
''',con=connection)\
.style.set_table_styles(style_dict)

,type,name,tbl_name,rootpage,sql
0,table,df1,df1,2,"CREATE TABLE ""df1"" ( ""key"" TEXT, ""value"" REAL )"
1,table,df2,df2,3,"CREATE TABLE ""df2"" ( ""key"" TEXT, ""value"" REAL )"
2,table,projects,projects,4,"CREATE TABLE projects ( id integer PRIMARY KEY, name text NOT NULL, begin_date text, end_date text)"
3,table,tasks,tasks,5,"CREATE TABLE tasks ( id integer PRIMARY KEY, name text NOT NULL, priority integer, status_id integer NOT NULL, project_id integer NOT NULL, begin_date text NOT NULL, end_date text NOT NULL, FOREIGN KEY (project_id) REFERENCES projects (id) )"
4,table,tips,tips,6,"CREATE TABLE ""tips"" ( ""index"" INTEGER, ""total_bill"" REAL, ""tip"" REAL, ""sex"" TEXT, ""smoker"" TEXT, ""day"" TEXT, ""time"" TEXT, ""size"" INTEGER )"
5,index,ix_tips_index,tips,7,"CREATE INDEX ""ix_tips_index""ON ""tips"" (""index"")"
6,table,schooledu,schooledu,11,"CREATE TABLE ""schooledu"" ( ""index"" INTEGER, ""LocationCode"" TEXT, ""District"" INTEGER, ""PercentELL"" REAL, ""PercentAsian"" REAL, ""PercentBlack"" REAL, ""PercentHispanic"" REAL, ""PercentBlack/Hispanic"" REAL, ""PercentWhite"" REAL, ""StudentAttendanceRate"" REAL, ""PercentofStudentsChronicallyAbsent"" REAL, ""RigorousInstruction%"" REAL, ""CollaborativeTeachers%"" REAL, ""SupportiveEnvironment%"" REAL, ""EffectiveSchoolLeadership%"" REAL, ""StrongFamily-CommunityTies%"" REAL, ""Trust%"" REAL, ""AverageELAProficiency"" REAL, ""AverageMathProficiency"" REAL, ""EconomicNeedIndex"" REAL, ""SchoolIncomeEstimate"" REAL, ""RigorousInstructionRating"" TEXT, ""CollaborativeTeachersRating"" TEXT, ""SupportiveEnvironmentRating"" TEXT, ""EffectiveSchoolLeadershipRating"" TEXT, ""StrongFamily-CommunityTiesRating"" TEXT, ""TrustRating"" TEXT, ""StudentAchievementRating"" TEXT )"
7,index,ix_schooledu_index,schooledu,12,"CREATE INDEX ""ix_schooledu_index""ON ""schooledu"" (""index"")"
8,table,nba,nba,55,"CREATE TABLE ""nba"" ( ""index"" INTEGER, ""Year"" INTEGER, ""Player"" TEXT, ""Pos"" TEXT, ""HT"" TEXT, ""WT"" INTEGER, ""Team"" TEXT, ""SelectionType"" TEXT, ""NBADraftStatus"" TEXT, ""Nationality"" TEXT )"
9,index,ix_nba_index,nba,56,"CREATE INDEX ""ix_nba_index""ON ""nba"" (""index"")"


In [22]:
if connection is not None:
    connection.close()
if os.path.exists('example.db'):
    os.remove('example.db')
else:
    print('The file does not exist')
os.listdir()

['__notebook__.ipynb']